In [1]:
import xml.etree.ElementTree as ET

def extract_text_from_xml(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    texts = []

    # Recursive function to extract all text within <text> elements
    def extract_text(element):
        if element.tag == "text":
            texts.append(element.text.strip() if element.text else "")
        for child in element:
            extract_text(child)
            if child.tail:
                texts.append(child.tail.strip())

    extract_text(root)

    return "\n".join(t for t in texts if t)  # Join all extracted texts

# Usage example
xml_file = "sidrac.xml"  # Replace with your XML file path
extracted_text = extract_text_from_xml(xml_file)
print(extracted_text)

Ende oec ute Ebreusche tale.
(
)
TAFEL
gevoelen ioie in hemelrike ende pine in der hellen.
sy ghemaect elcx daechs.
51 Welc es edelheit.
mensche in hem hebben mach.
78 Dwelke maect mere wijsheide inden mensche, soe coude spise soe heete.
waer af comen zij.
102 Eest goet dat men antwert den ghenen die scalckelijc ende sottelijc spreken.
Waer af compt die hagel.
133 Mochte iemen varen met goeden winde inde zee soe lange recht voert, datti quame daert tfirmament keert.
zande ende die dropele vander zee.
160 Welc es de meeste bliscap vander werelt.
haesten ofte niet haesten.
188 Welc es die donckerste ende die bedecste dinc diemen vint.
enege mere gracie in dander werelt.
welc es dan dbeste woert, dbeste cruit ende die beste steen daer die werelt by staet.
liede, die de sonden lieten omme Gode.
240 Sal een mensche gheloven wat hy peist vanden lieden.
267 Van wat dingen compt bliscap.
sy verloren soe behouden.
296 Doen Adam ghinc uten paradise vant hy te hant vrucht by te levene.
hoe mach h

In [2]:
import xml.etree.ElementTree as ET
import csv
import re

def extract_text_from_xml(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    texts = []

    # Recursive function to extract all text within <text> elements
    def extract_text(element):
        if element.tag == "text":
            texts.append(element.text.strip() if element.text else "")
        for child in element:
            extract_text(child)
            if child.tail:
                texts.append(child.tail.strip())

    extract_text(root)

    return " ".join(t for t in texts if t)  # Join all extracted texts

def clean_and_split_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    words = text.split()  # Split on spaces
    return words

def save_to_tsv(words, output_file):
    with open(output_file, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file, delimiter='\t')
        writer.writerow(["form", "pos", "lemma"])  # Write headers
        for word in words:
            writer.writerow([word, "", ""])  # Empty 'pos' and 'lemma' columns

# Usage example
xml_file = "sidrac.xml"  # Replace with your XML file path
output_tsv = "sidrac.tsv"

text = extract_text_from_xml(xml_file)
words = clean_and_split_text(text)
save_to_tsv(words, output_tsv)

print(f"TSV file saved as {output_tsv}")

TSV file saved as sidrac.tsv


In [27]:
import pandas as pd
from lxml import etree
from tqdm import tqdm

# Define input and output file paths
input_file = "sidrac.tsv"
output_file = "sidrac.xml"

# Function to convert TSV to XML
def convert_tsv_to_xml(tsv_filepath, xml_filepath):
    """Convert a TSV file into an XML document."""
    df = pd.read_csv(tsv_filepath, sep="\t", dtype=str).fillna("")  # Read TSV, fill empty values with ""

    root = etree.Element("text", id="sidrac")  # Root element <text id="sidrac">

    line_tokens, line_lemmas, line_words = [], [], []  # Buffers for line data
    line_number = 1  # Verse/line counter

    for _, row in tqdm(df.iterrows(), total=len(df), desc="Processing words"):
        form, pos, lemma = row["form"], row["pos"], row["lemma"]  # Extract columns

        if form == "$":  # End of a verse → Create <l> element
            if line_tokens:
                l_element = etree.SubElement(
                    root, "l",
                    n=str(line_number),
                    tokens=" ".join(line_tokens),
                    lemmas=" ".join(line_lemmas)
                )
                for w in line_words:
                    l_element.append(w)

                # Reset buffers for the next verse
                line_tokens, line_lemmas, line_words = [], [], []
                line_number += 1
        else:
            # Create <w> element
            w_element = etree.Element("w")
            form_el = etree.SubElement(w_element, "form")
            form_el.text = form

            lemma_el = etree.SubElement(w_element, "lemma")
            lemma_el.text = lemma

            pos_el = etree.SubElement(w_element, "pos")
            pos_el.text = pos

            # Append word data
            line_tokens.append(form)
            line_lemmas.append(lemma)
            line_words.append(w_element)

    # Save XML file
    tree = etree.ElementTree(root)
    tree.write(xml_filepath, encoding="utf-8", xml_declaration=True, pretty_print=True)

    print(f"Conversion complete! XML file saved as: {xml_filepath}")

# Run conversion
convert_tsv_to_xml(input_file, output_file)

Processing words: 100%|████████████████| 59531/59531 [00:00<00:00, 83480.12it/s]

Conversion complete! XML file saved as: sidrac.xml


In [12]:
import pandas as pd
from lxml import etree
from tqdm import tqdm

# Define input and output file paths
input_file = "sidrac.tsv"
output_file = "sidrac.xml"

# Function to convert TSV to XML
def convert_tsv_to_xml(tsv_filepath, xml_filepath):
    """Convert a TSV file into an XML document."""
    df = pd.read_csv(tsv_filepath, sep="\t", dtype=str).fillna("")  # Read TSV, fill empty values with ""

    root = etree.Element("text", id="sidrac")  # Root element <text id="sidrac">

    for _, row in tqdm(df.iterrows(), total=len(df), desc="Processing words"):
        form, pos, lemma = row["form"], row["pos"], row["lemma"]  # Extract columns

        # Create <w> element
        w_element = etree.SubElement(root, "w")

        form_el = etree.SubElement(w_element, "form")
        form_el.text = form

        lemma_el = etree.SubElement(w_element, "lemma")
        lemma_el.text = lemma

        pos_el = etree.SubElement(w_element, "pos")
        pos_el.text = pos

    # Save XML file
    tree = etree.ElementTree(root)
    tree.write(xml_filepath, encoding="utf-8", xml_declaration=True, pretty_print=True)

    print(f"Conversion complete! XML file saved as: {xml_filepath}")

# Run conversion
convert_tsv_to_xml(input_file, output_file)

Processing words: 100%|████████████████| 32677/32677 [00:00<00:00, 81526.78it/s]

Conversion complete! XML file saved as: sidrac.xml
